In [1]:
import torch
print("PyTorch has version {}".format(torch.__version__))

!pip install -q dgl-cu113 dglgo -f https://data.dgl.ai/wheels/repo.html

PyTorch has version 1.12.1+cu113
     |████████████████████████████████| 239.2 MB 1.2 kB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 280 kB 15.0 MB/s 
     |████████████████████████████████| 29.1 MB 1.5 MB/s 
     |████████████████████████████████| 45 kB 3.2 MB/s 
     |████████████████████████████████| 109 kB 48.4 MB/s 
     |████████████████████████████████| 78 kB 6.2 MB/s 
     |████████████████████████████████| 103 kB 45.7 MB/s 
     |████████████████████████████████| 52 kB 697 kB/s 
     |████████████████████████████████| 41 kB 228 kB/s 
     |████████████████████████████████| 3.2 MB 44.1 MB/s 
     |████████████████████████████████| 500 kB 44.1 MB/s 
     |████████████████████████████████| 133 kB 22.3 MB/s 
     |████████████████████████████████| 90 kB 6.7 MB/s 
     |████████████████████████████████| 1.1 MB 47.5 MB/s 
     |████████████████████████████████| 121 kB 50.7 MB/s 
     |████████████████████████████████| 84 kB 1.6 

In [2]:
import torch
import torch.nn as nn 
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import torch
import torch.nn as nn
import torch.nn.functional as F
import itertools
import numpy as np
import scipy.sparse as sp
import time
import dgl

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [3]:
import pandas as pd
train_df = pd.read_csv("/content/train.csv")
test_df = pd.read_csv("/content/test.csv")

In [4]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 7.5 MB/s 
     |████████████████████████████████| 182 kB 27.4 MB/s 
     |████████████████████████████████| 7.6 MB 48.8 MB/s 


In [5]:
import torch
from transformers import BertTokenizer, BertLMHeadModel,BertModel

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [6]:
tokenized_text = [tokenizer.tokenize(sen) for sen in train_df.claim.values] 

max_lenth = 0 
for i in tokenized_text:
  if len(i) >= max_lenth:
    max_lenth = len(i)

print(max_lenth)

96


In [7]:
from keras_preprocessing.sequence import pad_sequences

print('padding')
MAX_LEN = 100
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_text]
# input_ids = pad_sequences(input_ids, maxlen = MAX_LEN, dtype='long', truncating='post', padding='post')
input_ids[0]
print('input_ids[0]:',input_ids[0])

padding
input_ids[0]: [2106, 1037, 8839, 2450, 2695, 1037, 8257, 2055, 8398, 5026, 2067, 8864, 1029]


In [8]:
train_df ["bert"] = tokenized_text
train_df.head(3)

,id,label,claim,published_date,keybert_keywords,ner_keywords,youtube0,youtube1,youtube2,youtube3,youtube4,youtube5,youtube6,youtube7,youtube8,youtube9,bert
0,0,0,Did a Vermont Woman Post a Joke About Trump Br...,2018-01-25,"['trump', 'vermont', 'woman']","['vermont', 'trump', 'woman']",Citizen Trump A talk given at the University ...,News Donald Trump has a great woman problem N...,Trump Supporters Remain Despite Vulgar Slur J...,BREAKING Donald trump has a great woman proble...,Revolutionary War SNL The Patriots from New E...,World News Donald Trump has a great woman prob...,Latest News Donald Trump has a great woman pro...,AM Executive Session Gender Matters The U S Pr...,Trump Discusses Tax Bill Gives ;Total Support ...,Chicago s Women s March January This is what ...,"[did, a, vermont, woman, post, a, joke, about,..."
1,1,1,Were Baseball Players Photographed Kneeling to...,2017-11-08,"['lynchings', 'baseball', 's']","['lynchings', 'baseball', 's']",Tony Shalhoub Still ;Monk ; after all these ye...,Lesson Class differences and racial variation ...,Mass Media Mass Culture and the Golden Age of ...,Mark W Bennett quot;Implicit Bias and the Law ...,Chris Hedges Writing as Resistance presented b...,Race in America FOCIS th Anniversary lecture s...,,,,,"[were, baseball, players, photographed, kneeli..."
2,2,0,Wisconsin state Rep. John Nygren hits pay dirt...,2019-02-07,"['tax', 'wisconsin', 'dirt']","['rep', 'wisconsin', 'tax', 'wisconsin']",Pocan Shutdown Waste of Time Energy Pocan Shu...,Lawmaker This couldn ;t be more embarrassing f...,State Rep Bob Behning Creating teacher career ...,US GOV th,Representative John Macco of the th talks the ...,Democratic Senator Calls On Va Gov To Resign M...,Rep Sean Duffy introduces the Reciprocal Trade...,Rep Dan Caulkins on IL Minimum Wage Hike Stat...,Rep Weber Worried More Jobs Will Leave IL Rea...,Foxconn May Not Build B Wisconsin Plant Presid...,"[wisconsin, state, rep, ., john, ny, ##gren, h..."


In [258]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
def identity_tokenizer(text):
    return text


tfidf = TfidfVectorizer(tokenizer=identity_tokenizer, stop_words='english', lowercase=False ,max_features=2000)    
X_train_counts = tfidf.fit_transform(tokenized_text)
print(X_train_counts.shape)

from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
print(X_train_tfidf.shape)


(5277, 2000)
(5277, 2000)


In [13]:
#Trainset
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
count_vect = TfidfVectorizer(max_features=2000)
X_train_counts = count_vect.fit_transform(train_df.claim)
print(X_train_counts.shape)

from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
print(X_train_tfidf.shape)

(5277, 2000)
(5277, 2000)


In [14]:
#Testset 
from sklearn.feature_extraction.text import CountVectorizer
count_vect = TfidfVectorizer(max_features=2000)
X_test_counts = count_vect.fit_transform(test_df.claim)
print(X_test_counts.shape)

from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_test_tfidf = tfidf_transformer.fit_transform(X_test_counts)
print(X_test_tfidf.shape)

(1856, 2000)
(1856, 2000)


In [242]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, train_df.label, test_size = 0.2,shuffle=True )

In [243]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(4221, 2000)
(1056, 2000)
(4221,)
(1056,)


In [244]:
#Baseline : SVM

from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
# Train the SVM on the training set
svm = LinearSVC()
svm.fit(X_train,y_train)
# Obtain accuracy on the train set
y_hat = svm.predict(X_test)
acc = accuracy_score(y_hat, y_test)
print(f"Accuracy on the train set: {acc:.4f}")


Accuracy on the train set: 0.7027


In [248]:
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(solver='liblinear', multi_class='auto', max_iter=1000)

In [251]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
  
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}
grid = GridSearchCV(logreg, param_grid, cv=5, return_train_score=True)
grid.fit(X_train, y_train)
grid.score(X_test, y_test), grid.best_params_, grid.best_score_

(0.7225378787878788, {'C': 1}, 0.7199736391934715)

In [254]:
y_hat = grid.predict(X_test)
acc = accuracy_score(y_hat, y_test)
acc

0.7225378787878788

In [255]:
y_hat = grid.predict(X_test_tfidf)  
y_hat

array([0, 0, 0, ..., 1, 0, 0])

In [256]:
import pandas as pd
import numpy as np

# x_np = pred.numpy()
x_df = pd.DataFrame(y_hat)
x_df.to_csv('jh.csv')

In [40]:
#bert embedding 
!pip install tokenizers
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 5.0 MB/s 
     |████████████████████████████████| 182 kB 60.7 MB/s 


In [41]:
import torch
from transformers import BertTokenizer, BertLMHeadModel

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertLMHeadModel.from_pretrained("bert-base-uncased")

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs, labels=inputs["input_ids"])
loss = outputs.loss
logits = outputs.logits

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [112]:
tokenized_text = tokenizer.tokenize("Hello, my dog is cute")
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
indexed_tokens

[7592, 1010, 2026, 3899, 2003, 10140]

In [115]:
tokens_tensor = torch.tensor([indexed_tokens])
tokens_tensor

tensor([[ 7592,  1010,  2026,  3899,  2003, 10140]])

In [128]:
tokenized_text
tokenized_text = [tokenizer.tokenize(sen) for sen in train_df.claim.values] 

In [130]:
from keras_preprocessing.sequence import pad_sequences

print('padding')
MAX_LEN = 128
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_text]
input_ids = pad_sequences(input_ids, maxlen = MAX_LEN, dtype='long', truncating='post', padding='post')
input_ids[0]
print('input_ids[0]:',input_ids[0])

padding
input_ids[0]: [2106 1037 8839 2450 2695 1037 8257 2055 8398 5026 2067 8864 1029    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]


In [132]:
attention_masks = []
 
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)
 
print('attention_masks[0]:',attention_masks[0])

attention_masks[0]: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [15]:
##edge 연결 
import ast
keybert_list = [ast.literal_eval(i) for i in train_df.keybert_keywords]
print(len(keybert_list))

ner_list = [ast.literal_eval(i) for i in train_df.ner_keywords]
print(len(ner_list))

5277
5277


In [18]:
dic_array = np.zeros((len(train_df),len(train_df)))
print(dic_array.shape)
print(dic_array.sum())

for i, i_m in enumerate(keybert_list):
  for j, j_m in enumerate(keybert_list):
    for item in i_m:
      if item in j_m:
        dic_array[i][j] = 1 



(5277, 5277)
0.0
(5277, 5277)
0.0


In [19]:
dic_array = np.zeros((len(train_df),len(train_df)))
print(dic_array.shape)
print(dic_array.sum())

for i, i_m in enumerate(ner_list):
  for j, j_m in enumerate(ner_list):
    for item in i_m:
      if item in j_m:
        dic_array[i][j] = 1 

print(dic_array.shape)
print(dic_array.sum())

for i, i_m in enumerate(keybert_list):
  for j, j_m in enumerate(keybert_list):
    for item in i_m:
      if item in j_m:
        dic_array[i][j] = 1 


print(dic_array.shape)
print(dic_array.sum())

(5277, 5277)
0.0
(5277, 5277)
1328495.0
(5277, 5277)
1382295.0


In [20]:
adj_t = torch.nonzero(torch.tensor(dic_array)).T
print("== adj to edge list: \n", adj_t)

== adj to edge list: 
 tensor([[   0,    0,    0,  ..., 5276, 5276, 5276],
        [   0,    5,    7,  ..., 5161, 5176, 5276]])


In [21]:
u,v = adj_t[0], adj_t[1]
g = dgl.graph((u, v))
g

Graph(num_nodes=5277, num_edges=1382295,
      ndata_schemes={}
      edata_schemes={})

In [22]:
feature = X_train_tfidf.toarray()

In [23]:
feature.shape

(5277, 2000)

In [24]:
g.ndata['features'] = torch.tensor(feature).float()

In [25]:
##label 달기
labels = train_df.label.values
print(len(labels))

g.ndata['label'] = torch.tensor(labels)
g

5277


Graph(num_nodes=5277, num_edges=1382295,
      ndata_schemes={'features': Scheme(shape=(2000,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={})

In [26]:
nids = np.random.permutation(np.arange(g.number_of_nodes()))
test_size = int(len(nids) * 0.2)
test_id = nids[:test_size]

train_id = nids[test_size:]
val_size = int(len(train_id) * 0.1)

val_id = train_id[:val_size]
train_id = train_id[val_size:]


print("train_size: ", len(train_id), "test_size: ", len(test_id),"val_size: ", len(val_id) )

train_size:  3800 test_size:  1055 val_size:  422


In [175]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F

from dgl.nn import GraphConv


class GCN(nn.Module):
    def __init__(self, in_feats, h_feats,num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h


from dgl.nn import SAGEConv

# build a two-layer GraphSAGE model
class GraphSAGE(nn.Module):
    def __init__(self, in_feats, h_feats,num_classes):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats, 'mean')
        # self.conv2 = SAGEConv(h_feats, h_feat, 'mean')
        self.conv3 = SAGEConv(h_feats, num_classes, 'mean')
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        # h = F.relu(h)
        # h = self.dropout(h)
        # h = self.conv2(g, h)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv3(g, h)
        return h




In [207]:
# model = GCN(g.ndata['features'].shape[1], 64, 2)
model = GraphSAGE(g.ndata['features'].shape[1], 128,2)
model

GraphSAGE(
  (conv1): SAGEConv(
    (feat_drop): Dropout(p=0.0, inplace=False)
    (fc_self): Linear(in_features=2000, out_features=128, bias=False)
    (fc_neigh): Linear(in_features=2000, out_features=128, bias=False)
  )
  (conv3): SAGEConv(
    (feat_drop): Dropout(p=0.0, inplace=False)
    (fc_self): Linear(in_features=128, out_features=2, bias=False)
    (fc_neigh): Linear(in_features=128, out_features=2, bias=False)
  )
  (dropout): Dropout(p=0.5, inplace=False)
)

In [208]:
def train(g,features,labels,train_mask,val_mask):
    model.train()
    optimizer =  torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0


    logits = model(g, features)
    loss = F.cross_entropy(logits[train_mask], labels[train_mask])
    
    

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    pred = logits.argmax(1)
    
    train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
    val_acc = (pred[val_mask] == labels[val_mask]).float().mean()

    if best_val_acc < val_acc:
        best_val_acc = val_acc

    return logits, loss, pred, train_acc, val_acc, best_val_acc

In [209]:
from sklearn.metrics import classification_report

def test(g,features,labels,test_mask):
  with torch.no_grad():
    model.eval()

    answer=[]

    logits = model(g, features)
    loss = F.cross_entropy(logits[test_mask], labels[test_mask])


    pred = logits.argmax(1)
    test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

    print(classification_report(pred[test_mask],labels[test_mask]))


    return logits, loss, pred, test_acc

In [210]:
def main(epoch, g, model,train_id, test_id,val_id): 

  features = g.ndata['features']
  labels = g.ndata['label']
  train_mask = train_id
  test_mask = test_id
  val_mask = val_id


  # train
  for e in range(epoch):
    logits, loss, pred, train_acc, val_acc, best_val_acc = train(g,features,labels,train_mask,val_mask)
    print('In epoch {}, loss: {:.3f}, train acc: {:.3f}, val acc: {:.3f} (best {:.3f})'.format(
                e, loss,train_acc, val_acc, best_val_acc))


  # test
  logits, loss, pred, test_acc = test(g,features,labels,test_mask)
  print('[Test result] loss: {:.3f}, test acc: {:.3f}'.format(loss, test_acc))

  


In [211]:
g = dgl.add_self_loop(g)

In [212]:
main(35,g,model,train_id, test_id, val_id)

In epoch 0, loss: 0.697, train acc: 0.497, val acc: 0.512 (best 0.512)
In epoch 1, loss: 0.624, train acc: 0.622, val acc: 0.573 (best 0.573)
In epoch 2, loss: 0.586, train acc: 0.668, val acc: 0.576 (best 0.576)
In epoch 3, loss: 0.555, train acc: 0.680, val acc: 0.597 (best 0.597)
In epoch 4, loss: 0.520, train acc: 0.715, val acc: 0.597 (best 0.597)
In epoch 5, loss: 0.498, train acc: 0.739, val acc: 0.621 (best 0.621)
In epoch 6, loss: 0.471, train acc: 0.760, val acc: 0.640 (best 0.640)
In epoch 7, loss: 0.454, train acc: 0.777, val acc: 0.640 (best 0.640)
In epoch 8, loss: 0.433, train acc: 0.794, val acc: 0.640 (best 0.640)
In epoch 9, loss: 0.417, train acc: 0.809, val acc: 0.647 (best 0.647)
In epoch 10, loss: 0.402, train acc: 0.807, val acc: 0.630 (best 0.630)
In epoch 11, loss: 0.391, train acc: 0.822, val acc: 0.661 (best 0.661)
In epoch 12, loss: 0.376, train acc: 0.825, val acc: 0.659 (best 0.659)
In epoch 13, loss: 0.365, train acc: 0.843, val acc: 0.649 (best 0.649)
In

In [188]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = TfidfVectorizer(max_features=2000)
X_test_counts = count_vect.fit_transform(test_df.claim)
print(X_test_counts.shape)

from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_test_tfidf = tfidf_transformer.fit_transform(X_test_counts)
print(X_test_tfidf.shape)


##edge 연결 
import ast
keybert_list = [ast.literal_eval(i) for i in test_df.keybert_keywords]
print(len(keybert_list))

ner_list = [ast.literal_eval(i) for i in test_df.ner_keywords]
print(len(ner_list))

dic_array = np.zeros((len(test_df),len(test_df)))
print(dic_array.shape)
print(dic_array.sum())

for i, i_m in enumerate(ner_list):
  for j, j_m in enumerate(ner_list):
    for item in i_m:
      if item in j_m:
        dic_array[i][j] = 1 

print(dic_array.shape)
print(dic_array.sum())

for i, i_m in enumerate(keybert_list):
  for j, j_m in enumerate(keybert_list):
    for item in i_m:
      if item in j_m:
      # dic_array[i][j] += 1 # if edge weight를 적용해보고 싶다면 frequency 활용해볼 수도 
        dic_array[i][j] = 1 # 필요없다면


print(dic_array.shape)
print(dic_array.sum())

adj_t = torch.nonzero(torch.tensor(dic_array)).T
print("== adj to edge list: \n", adj_t)

u,v = adj_t[0], adj_t[1]
g_test = dgl.graph((u, v))
g_test

feature = X_test_tfidf.toarray()
g_test.ndata['features'] = torch.tensor(feature).float()
##label 달기

g_test = dgl.add_self_loop(g_test)

(1856, 2000)
(1856, 2000)
1856
1856
(1856, 1856)
0.0
(1856, 1856)
828444.0
(1856, 1856)
883516.0
== adj to edge list: 
 tensor([[   0,    0,    0,  ..., 1855, 1855, 1855],
        [   0,   40,   82,  ..., 1850, 1854, 1855]])


In [189]:
with torch.no_grad():
  model.eval()

  logits = model(g_test, g_test.ndata['features'])
  
  pred = logits.argmax(1)
  print(pred)
  

tensor([1, 1, 0,  ..., 1, 1, 1])


In [190]:
import torch
import pandas as pd
import numpy as np

x_np = pred.numpy()
x_np = [0 if i ==1 else 1 for i in x_np]
x_df = pd.DataFrame(x_np)
x_df.to_csv('jh.csv')